# Model Explainability

This notebook demonstrates:
- Permutation importance
- SHAP (TreeExplainer, summary/waterfall/dependence plots)
- LIME (Local explanations)
- Partial Dependence Plots (PDP) and ICE
- SHAP vs LIME comparison

**Requirements**: `pip install shap lime`

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.inspection import permutation_importance, PartialDependenceDisplay

# Load data and train model
housing = fetch_california_housing(as_frame=True)
X, y = housing.data, housing.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

gb = GradientBoostingRegressor(n_estimators=200, max_depth=5, random_state=42)
gb.fit(X_train, y_train)
print(f'Test R²: {gb.score(X_test, y_test):.4f}')

## 1. Permutation Importance

In [ ]:
# Why: Permutation importance measures the actual impact on test performance when a
# feature's values are shuffled — unlike built-in feature_importances_ (which reflect
# training split frequency), permutation importance is model-agnostic and computed on
# held-out data, making it a more honest measure of predictive contribution.
perm = permutation_importance(gb, X_test, y_test, n_repeats=10, random_state=42, n_jobs=-1)
perm_imp = pd.Series(perm.importances_mean, index=X_train.columns).sort_values(ascending=True)

fig, ax = plt.subplots(figsize=(8, 5))
perm_imp.plot(kind='barh', xerr=perm.importances_std, ax=ax, color='coral')
ax.set_title('Permutation Importance')
ax.set_xlabel('Decrease in R²')
plt.tight_layout()
plt.show()

## 2. SHAP Analysis

In [ ]:
import shap

# Why: TreeExplainer uses the tree structure to compute exact SHAP values in polynomial
# time (vs. exponential for KernelSHAP), making it the preferred explainer for tree-based
# models. SHAP values satisfy additivity: base_value + sum(shap_values) = prediction.
explainer = shap.TreeExplainer(gb)
shap_values = explainer.shap_values(X_test)

# Verify additivity
pred = gb.predict(X_test.iloc[[0]])[0]
shap_sum = explainer.expected_value + shap_values[0].sum()
print(f'Prediction: {pred:.4f}, SHAP sum: {shap_sum:.4f}, diff: {abs(pred-shap_sum):.6f}')

In [ ]:
# Summary plot (global)
shap.summary_plot(shap_values, X_test)

In [ ]:
# Waterfall plot (local - single prediction)
shap.plots.waterfall(shap.Explanation(
    values=shap_values[0],
    base_values=explainer.expected_value,
    data=X_test.iloc[0],
    feature_names=X_test.columns.tolist()
))

In [ ]:
# Dependence plot
shap.dependence_plot('MedInc', shap_values, X_test, interaction_index='AveOccup')

## 3. LIME

In [ ]:
import lime
import lime.lime_tabular

# Why: LIME builds a local linear approximation around each individual prediction,
# making it interpretable for any black-box model. Unlike SHAP (global consistency),
# LIME focuses on local fidelity — the explanation is faithful in the neighborhood
# of this specific instance, not necessarily globally.
lime_explainer = lime.lime_tabular.LimeTabularExplainer(
    training_data=X_train.values,
    feature_names=X_train.columns.tolist(),
    mode='regression'
)

explanation = lime_explainer.explain_instance(
    X_test.iloc[0].values, gb.predict, num_features=8
)

print(f'Prediction: {gb.predict(X_test.iloc[[0]])[0]:.4f}')
print('\nLIME contributions:')
for feat, weight in explanation.as_list():
    print(f'  {feat}: {weight:+.4f}')

explanation.as_pyplot_figure()
plt.tight_layout()
plt.show()

## 4. Partial Dependence Plots (PDP + ICE)

In [ ]:
# Why: kind='both' overlays Individual Conditional Expectation (ICE) curves on the PDP.
# ICE lines reveal heterogeneous effects hidden by the average PDP — if ICE lines
# diverge, the feature's effect varies across subpopulations (interaction effects).
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
PartialDependenceDisplay.from_estimator(
    gb, X_test.iloc[:200], ['MedInc', 'AveOccup', 'Latitude'],
    kind='both',
    ax=axes,
    ice_lines_kw={'alpha': 0.1, 'color': 'steelblue'},
    pd_line_kw={'color': 'red', 'linewidth': 2},
    n_jobs=-1
)
plt.suptitle('ICE + PDP', fontsize=14)
plt.tight_layout()
plt.show()

## 5. SHAP vs LIME Comparison

In [ ]:
# Compare explanations for the same prediction
sample_idx = 0

shap_exp = pd.Series(shap_values[sample_idx], index=X_test.columns, name='SHAP')

lime_exp = lime_explainer.explain_instance(X_test.iloc[sample_idx].values, gb.predict, num_features=8)
lime_dict = {feat.split(' ')[0]: w for feat, w in lime_exp.as_list()}
lime_series = pd.Series(lime_dict, name='LIME')

comparison = pd.DataFrame({'SHAP': shap_exp, 'LIME': lime_series}).fillna(0)
comparison.plot(kind='barh', figsize=(10, 6))
plt.title('SHAP vs LIME')
plt.xlabel('Feature Contribution')
plt.axvline(0, color='black', linewidth=0.5)
plt.tight_layout()
plt.show()